In [148]:
lines = []
with open("aug_27_6pmMDT.txt") as in_file:
    for line in in_file:
        lines.append(line.strip())

In [149]:
kw = [(y[0],int(y[-1])) for y in [x.split(":") for x in lines[3:-1]]]

In [150]:
harv_kw_sum = 0;
for k in kw:
    x = k[0].split()
    if len(x) > 1:
        if "harvey" not in x:
            harv_kw_sum += k[1]
    else:
        harv_kw_sum += k[1]
harv_kw_sum

5880488

# Geo Tag?

In [151]:
import json, sys, pprint
import pandas as pd

In [152]:
geo_tagged = []
count = 0;
with open('tweets_august_28.jsonl') as inFile:
    for line in inFile:
        count += 1
        try:
            t = json.loads(line.strip())
            if 'coordinates' in t:
                if t['coordinates']:
                    geo_tagged.append(t)
                    sys.stderr.write("\rFound {0} geotagged of {1} processed".format(len(geo_tagged),count))
        except:
            pprint.pprint(line.strip())
            continue
len(geo_tagged)  

Found 5685 geotagged of 4547313 processed

5685

In [155]:
def tweet_to_feature(t):
    return {
        "type":"Feature",
        "geometry" : {"type" : "Point", "coordinates" : t['coordinates']['coordinates']},
        "properties":{
            "created_at" : t['created_at'],
            "timestamp"  : int(pd.Timestamp(t['created_at']).timestamp()),
            "text" : t['text'],
            "user" : t['user']['screen_name']
        }
    }

In [156]:
with open('/data/www/jennings/harvey.geojson','w') as outFile:
    ts = []
    for t in geo_tagged:
        ts.append(tweet_to_feature(t))
    json.dump({"type" : "FeatureCollection","features" : ts}, outFile)